## Implementation using CVX


In [3]:
import cvxpy
import numpy as np

In [306]:
network = 'data/braess'
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
nb_links = graph.shape[0]
first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
graph[:,1] = graph[:,1]-first_index_od
graph[:,2] = graph[:,2]-first_index_od
demand[:,0] = demand[:,0] - first_index_od
demand[:,1] = demand[:,1] - first_index_od

In [307]:
f_cvx = cvxpy.Variable(nb_links)

obj_cvx = np.sum(graph[:,3]*f_cvx + 1/2*graph[:,4]*(f_cvx**2) + 1/3*graph[:,5]*(f_cvx**3) + 1/4*graph[:,6]*(f_cvx**4) + 1/5*graph[:,7]*(f_cvx**5))

### TO DO: Define delta (incidence matrix), the paths, A and u

In [316]:
h_cvx = cvxpy.Variable((delta.shape[0]))
print(f_cvx.shape)
print(h_cvx.shape)
print(delta.T.shape)
print(delta.T @ h_cvx)
u = [10000000 for _ in range(nb_links)]

(5,)
(3,)
(5, 3)
[[1. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 1.]] * var3652


In [317]:
constr = [
    f_cvx >= delta.T @ h_cvx,
    h_cvx >= 0,
    sum(h_cvx) >= demand[0][2]
    # (route2od+1) @ h_cvx == demand[:,2],
    # f_cvx <= u
]

In [380]:
pb = cvxpy.Problem(cvxpy.Minimize(graph[:,3]*f_cvx + + 1/2*graph[:,7]*(cvxpy.power(f_cvx,2))), constr)

In [381]:
pb.solve()

225.0000218751541

In [382]:
h_sol = h_cvx.value
f_sol = f_cvx.value
print(delta.T @ h_sol)
print((route2od+1) @ h_sol )

[9.99999956e+01 5.38155839e-06 9.99999902e+01 5.38155843e-06
 9.99999956e+01]
100.00000101162236


In [383]:
print(demand[0][2])

100.0


In [384]:
print(f_cvx.value)

[9.99999974e+01 6.25995712e-06 1.00000008e+02 6.25995716e-06
 9.99999974e+01]


## Annex to compute the paths from the links

In [312]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

In [441]:
from collections import defaultdict 
   
#This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
          
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
  
        # If current vertex is same as destination, then print 
        # current path[] 
        if u ==d: 
            # print(path)
            path_tmp = np.zeros(shape=(nb_links))
            for i in range(len(path)-1):
                path_tmp[graph_dict[path[i]][path[i+1]]] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, nb_links))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, nb_links)), axis=0)
        else: 
            # If current vertex is not destination 
            #Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path) 
                      
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop() 
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d,visited, path) 
        return self.paths_m 

# argument graph
def delta_matrix():
    route2od = np.array([])
    paths_matrix = np.array([])
    g = Graph(nb_links) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od

In [442]:
paths_matrix, route2od = delta_matrix()
print(paths_matrix)
delta = paths_matrix

[[1. 0. 1. 0. 1.]
 [1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 1.]]


In [443]:
print(paths_matrix.shape)
print(route2od)

(3, 5)
[0. 0. 0.]
